In [1]:
!nvidia-smi

Mon Sep 19 17:02:27 2016       
+------------------------------------------------------+                       
| NVIDIA-SMI 352.39     Driver Version: 352.39         |                       
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  Off  | 0000:04:00.0     Off |                  N/A |
| 35%   76C    P2   218W / 250W |   7636MiB / 12287MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX TIT...  Off  | 0000:05:00.0     Off |                  N/A |
| 44%   82C    P2   224W / 250W |  10643MiB / 12287MiB |     88%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
from __future__ import print_function

from operator import mul
import os
import sys

import h5py
import ipyparallel
import numpy as np

# import PyGreentea as pygt

In [4]:
class NetWorker(object):
    def __init__(self, net_proto_path, caffemodel_path, gpu_device=0):
        self.net_proto_path = net_proto_path
        self.caffemodel_path = caffemodel_path
        self.gpu_device = gpu_device
    
    def process(self, dataset):
        import PyGreentea as pygt
        pygt.caffe.set_mode_gpu()
        pygt.caffe.set_device(self.gpu_device)
        net = pygt.caffe.Net(net_path, caffemodel_path, pygt.caffe.TEST)
        preds = pygt.process(net, [dataset])
        return preds

In [9]:
net_path = "/groups/turaga/home/grisaitisw/experiments/run_0912_1/net_test_big.prototxt"
caffemodel_path = "/groups/turaga/home/grisaitisw/experiments/run_0912_1/net_iter_20000.caffemodel"
test_device = 7
net_worker = NetWorker(net_path, caffemodel_path, test_device)

In [6]:
client = ipyparallel.Client("/groups/turaga/home/grisaitisw/.ipython/profile_greentea/security/ipcontroller-client.json")

In [8]:
lbv = client.load_balanced_view()

In [36]:
def process_func(net_path, caffemodel_path, dataset):
    import h5py
    import PyGreentea as pygt
    data_filename = dataset["data"]
    data_key = "main"
    with h5py.File(data_filename, "r") as h5_file:
        data_h5_dataset = h5_file[data_key]
        dataset["data"] = data_h5_dataset
        pygt.caffe.set_mode_gpu()
        pygt.caffe.set_device(self.gpu_device)
        net = pygt.caffe.Net(net_path, caffemodel_path, pygt.caffe.TEST)
        preds = pygt.process(net, [dataset])
        return True

In [37]:
dataset = dict(
    data="/groups/turaga/home/turagas/data/FlyEM/fibsem_medulla_7col/trvol-250-1-h5/im_uint8.h5",
    image_scaling_factor=0.5**8,
    name="trvol-250-1-h5"
)
#     preds = net_worker.process(dataset)
#     async_result = lbv.apply_async(net_worker.process, dataset)
async_result = lbv.apply_async(process_func, net_path, caffemodel_path, dataset)

In [ ]:
async_result.get()

In [ ]:
async_result = lbv.apply_async

In [ ]:
pred = preds[0]
print(pred.dtype, pred.shape, np.mean(pred))

In [ ]:
import multiprocessing

In [ ]:
process = multiprocessing.Process(target=net_worker.process, args=(d))

In [ ]:
random_state = np.random.RandomState(seed=0)
shape = (1,) + (148,) * 3
size = reduce(mul, shape)
input_data = random_state.uniform(size=size).astype(np.float32)
input_data = input_data.reshape(shape)
print(input_data.dtype, input_data.shape, input_data.mean())
dummy_slice = np.ascontiguousarray([0]).astype(np.float32)